#### Загрузка библиотек

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
from pathlib import Path
import re
from copy import deepcopy
from tqdm import tqdm
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM #, GenerationConfig


In [3]:
MAX_TOKEN_COUNT = 5000
%env CUDA_VISIBLE_DEVICES=3
BOXED_PATTERN = r'\\boxed\{(.*?)\}'

env: CUDA_VISIBLE_DEVICES=3


### Подготовка данных

In [3]:
linal = {
    r"Decompose this vector $x\in X$ according to the basis $\{e_{i}\}_{i=1}{n}$ of \
the linear space $X$. \\ \
 \
\begin{equation*} \
x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{1} = \begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{equation*}" : r"[73/24, -2, -19/24]",

r"Find some basis for the space $X$ defined as a linear shell \
of vectors $\{x_{i}\}_{i=1}^{m}$. \
 \
\begin{gather*} \
  x_{1} = \begin{pmatrix} -1 & 4 & -3 & -2 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{2} = \begin{pmatrix} 3 & -7 & 5 & 3 \end{pmatrix}^{\mathrm{T}}, \\ \
  x_{3} = \begin{pmatrix} 3 & -2 & 1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  x_{4} = \begin{pmatrix} -4 & 1 & 0 & 1 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}" : r"\begin{gather*} \
        \begin{pmatrix} -1 & 4 & -3 & -2 \end{pmatrix}^{\mathrm{T}}, \quad \
        \begin{pmatrix} 0 & 5 & -4 & -3 \end{pmatrix}^{\mathrm{T}}, \quad \
        \begin{pmatrix}  0 & 0 & -4 & 9 \end{pmatrix}^{\mathrm{T}}. \
    \end{gather*}",


r"Find the basis of the intersection of subspaces $L_{1}$ and $L_{2}$ of a linear \
space $X$ spanned by the corresponding vector systems $\{x_{i}\}_{i=1}^{p}$ and \
$\{y_{j}\}_{j=1}^{q}$ respectively. \
 \
\begin{gather*} \
  L_{1} = \left\langle (1,2,3,1,1)^\mathrm{T}, (1,0,1,-2,-2)^\mathrm{T}, \
    (2,0,1,-1,0)^\mathrm{T},(0,1,1,0,0)^\mathrm{T} \right\rangle, \\ \
  L_{2} = \left\langle (1,2,0,0,2)^\mathrm{T}, (0,1,-2,3,-3)^\mathrm{T}, \
    (-1,2,1,2,0)^\mathrm{T}, (1,1,-2,0,0)^\mathrm{T} \right\rangle. \
\end{gather*}" : r"$(0,0,1,0,0)^\mathrm{T}$, $(0,72,0,43,0)^\mathrm{T}$, $(18,-0,0,55,-76)^\mathrm{T}$.",

r"Find the coefficients of the linear form $f\in X^{\ast}$ in a given basis \
$\{e_{i}\}_{i=1}{n}$ spaces $X$. \
 \
\begin{gather*} \
f(x) = 4 \xi^{1} + 2 \xi^{2} + \xi^{3}, \\ \
e_{1} =\begin{pmatrix} 5 & 0 & 4 \end{pmatrix}^{\math{T}}, \quad \
  e_{2} = \begin{pmatrix} 5 & -1 & 0 \end{pmatrix}^{\mathrm{T}}, \quad \
  e_{3} = \begin{pmatrix} -1 & 0 & 4 \end{pmatrix}^{\mathrm{T}}. \
\end{gather*}" : r"(24, 18, 0)",

r"Find the basis of a linear shell $L$ given by a system of linear \
algebraic equations. \
 \
\begin{gather*} \
  \left\{ \begin{array}{lcr} \
    x_1 - x_2 + x_3 - x_4 &=& 0; \\ \
    x_1 + x_2 + 2x_3 + 3x_4 &=& 0;\\ \
    2x_1 + 4x_2 + 5x_3 + 10x_4 &=& 0;\\ \
    2x_1 - 4x_2 + x_3 - 6x_4 &=& 0; \
  \end{array} \right. \
\end{gather*}": r"$(3, 1, -2, 0)^\mathrm{T}$, $(5, 0, -4, 1)^\mathrm{T}$", 

r"Find the coordinates of the vector $x$ of the linear space $X$ in the basis \
$\{\$\{\tilde e_{j}\}_{j=1}^{n}$ if its coordinates in the basis are known \
$\{e_{i}\}_{i=1}^{n}$. \
 \
\begin{gather*} \
  x = \begin{pmatrix} 6 & 2 & 9 \end{pmatrix}^{\mathrm{T}}, \\ \
  \tilde e_{1} = 5e_{1} + 4e_{3}, \quad \
  \tilde e_{2} = 5e_{1} - e_{2} , \quad \
  \tilde e_{3} = -e_{1} + 4e_{3}. \
\end{gather*}" : r"$(\frac{73}{24}, -2, -\frac{19}{24})^\mathrm{T}$"
}
linal_df = []
for k, v in linal.items():
    task = {}
    task["subfield"] = "GeolinLinal"
    task["question"] = r"Please write valid Lean code to solve the problem using the Lean engine: \n" + k
    task["final_answer"] =  "['$" + v + "$']"
    task["solution"] = None
    task["solution_length"] = None
    linal_df.append(task)

linal_df = pd.DataFrame(linal_df)
linal_df.index = np.arange(0, len(linal_df))

linal_df.to_csv("/home/anmilka/different/linal_df.csv")

linal_dict = linal_df.T.to_dict().values()
print(len(linal_dict))


6


In [4]:
linal_df.head()

,subfield,question,final_answer,solution,solution_length
0,GeolinLinal,Please write valid Lean code to solve the prob...,"['$[73/24, -2, -19/24]$']",None,None
1,GeolinLinal,Please write valid Lean code to solve the prob...,['$\begin{gather*} \\n \begin{pmatrix} ...,None,None
2,GeolinLinal,Please write valid Lean code to solve the prob...,"['$$(0,0,1,0,0)^\mathrm{T}$, $(0,72,0,43,0)^\m...",None,None
3,GeolinLinal,Please write valid Lean code to solve the prob...,"['$(24, 18, 0)$']",None,None
4,GeolinLinal,Please write valid Lean code to solve the prob...,"['$$(3, 1, -2, 0)^\mathrm{T}$, $(5, 0, -4, 1)^...",None,None


In [5]:
def find_boxed_content(text):
    start_index = text.rfind(r'\boxed{')
    if start_index == -1:
        return None
    start_index += len(r'\boxed{')    
    stack = ['{']
    content = []

    for i in range(start_index, len(text)):
        char = text[i]
        content.append(char)
        if char == '{':
            stack.append(char)
        elif char == '}':
            if stack:
                stack.pop()
            if not stack:
                return ''.join(content[:-1])
    return None

### deepseek-math-7b-instruct

In [6]:
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_Fast=False)
#model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question'] +  ' Please reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": question}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_TOKEN_COUNT)

    detailed_result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)

    result = find_boxed_content(detailed_result)

    task['deepseek-math_detailed_result'] = detailed_result
    task['deepseek-math_result'] = result

pd.DataFrame(linal_dict).to_csv("/home/anmilka/different/lean_output/linal_deepseek.csv")

2025-02-04 14:54:02.894551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 14:54:02.894626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 14:54:02.896870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 14:54:02.908171: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-04 14:54:04.539896: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1it [01:20, 80.58s/it]This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
6it [08:45, 87.66s/it] 


### Qwen2.5-Math-7B CoT

In [7]:
model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question']

    # CoT
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": question}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=MAX_TOKEN_COUNT)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    detailed_result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    result = find_boxed_content(detailed_result)

    task['Qwen2.5-cot_detailed_result'] = detailed_result
    task['Qwen2.5-cot_result'] = result

pd.DataFrame(linal_dict).to_csv("/home/anmilka/different/lean_output/linal_qwen_cot.csv")


model-00001-of-00004.safetensors:  42%|####1     | 1.66G/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

6it [08:06, 81.08s/it]


### Qwen2-Math-7B TIR

In [8]:
for _, task in tqdm(enumerate(linal_dict)):
    question = task['question']

    messages = [
        {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."},
        {"role": "user", "content": question}
    ]
 
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=MAX_TOKEN_COUNT)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    detailed_result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    result = find_boxed_content(detailed_result)

    task['Qwen2.5-tir_detailed_result'] = detailed_result
    task['Qwen2.5-tir_result'] = result

pd.DataFrame(linal_dict).to_csv("/home/anmilka/different/lean_output/linal_qwen_tir.csv")

6it [06:57, 69.55s/it]


### DeepSeek-r1

In [9]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

for _, task in tqdm(enumerate(linal_dict)):
    question = task['question']

    # CoT
    messages = [
        {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
        {"role": "user", "content": question}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=MAX_TOKEN_COUNT)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    detailed_result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    result = find_boxed_content(detailed_result)

    task['DeepSeek-r1-Qwen_detailed_result'] = detailed_result
    task['DeepSeek-r1-Qwen_result'] = result

pd.DataFrame(linal_dict).to_csv("/home/anmilka/different/lean_output/linal_DeepSeek-r1-Qwen.csv")


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
1it [01:43, 103.52s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
2it [07:15, 237.85s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
3it [12:50, 282.04s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
4it [18:20, 301.28s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
5it [20:51, 246.81s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
6it [22:36, 226.04s/it]
